In [1]:
import os
import sqlite3 as db

import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Get PATH folder from previous directorys
def get_path(prev_folders:int=0):
    for i in range(prev_folders-1): os.chdir('../')  # Change to previous folder
    PATH = os.path.dirname(os.getcwd()) + '/'
    PATH = PATH.replace('\\', '/')
    return PATH

In [3]:
def get_product_type(product_model:str):
  product_model = product_model.lower().replace('-', ' ')
  labels = ('mountain', 'road', 'touring', 'classic')

  if any(word in product_model for word in labels):
    list_prov = product_model.split()

    for i in range(len(list_prov)):

        info_dict = {
           'mountain': 'Mountain',
           'road':'Road',
           'touring':'Touring',
           'classic':'Classic'
           }
        
        return info_dict.get(list_prov[i], 'Ninguno')

# Environment settings

In [4]:
pd.options.display.max_columns = None  # Remove "dots" from display when printing dataframes
PATH = get_path(prev_folders=2)

# ETL

## Read data

In [5]:
filepaths = [f for f in os.listdir(PATH + 'data/CREDITO') if f.endswith('.csv')]  # Get filenames

# Generate dataframe names
df_names = filepaths.copy()
for i in range(len(df_names)): df_names[i] = 'df_' + df_names[i].rstrip('.csv')

In [6]:
# Read multiples CSV
for i in range(len(df_names)):
    name = df_names[i]
    globals()[name] = pd.read_csv(  # Convert string into variable name with "globals()[variable_name]"
        PATH + 'data/CREDITO/' + filepaths[i],
        encoding='latin-1'
        )

## Load data to SQLite

In [7]:
connection = db.connect('Punto 1/CREDITO.db')

In [8]:
for i in range(len(df_names)):
    dataframe = df_names[i]
    table_name = df_names[i].lstrip('df_')
    globals()[dataframe].to_sql(  # Convert string into variable name with "globals()[variable_name]"
        table_name,
        connection,
        if_exists='replace',
        index=False
    )

In [9]:
del filepaths, df_names  # Delete variables to free memory

# First part

Cargar en una base de datos los archivos de CREDITO que se adjuntan, para todos los puntos adjuntar scripts, resultados y la evidencia que considere.

## 1.a

Unir las tablas que cargo de forma separada en una única base y nómbrela con sus iniciales.

In [10]:
amt = pd.concat(
    [df_CREDITO_201508, df_CREDITO_201509, df_CREDITO_201510, df_CREDITO_201511, df_CREDITO_201512],
    axis=0,
    ignore_index=True
    )

In [11]:
amt.to_sql(
        'AMT',  # Table name
        connection,
        if_exists='replace',
        index=False
    )

10000

Haga un conteo de Número de Créditos únicos.

In [12]:
len(amt['NumCre'].unique().tolist())

5910

Realice una suma de la columna ValPre por periodo.

In [13]:
amt.groupby('PERIODO')['ValPre'].sum().reset_index()

,PERIODO,ValPre
0,201508,8968671422
1,201509,14573857471
2,201510,11353207322
3,201511,33703517567
4,201512,8407501137


Determine por la columna NomLin el valor de los prestamos asignados para el periodo 201508

In [14]:
df_new = amt.query("PERIODO == 201508")
df_new.groupby('NomLin')['ValPre'].sum().reset_index()

,NomLin,ValPre
0,CAMPAÑA FECHAS ESPECIALES,405002541
1,CAPITAL DE TRABAJO MICROCREDITO,147788165
2,COMPRA DE CARTERA,29490000
3,COMPRA DE CARTERA ESPECIAL,17992400
4,CONVENIO GRANDES SUPERFICIES,696403
5,EDUCACION,20910981
6,FERIA DE CREDITO COMPENSAR,832899573
7,FERIA DE TURISMO,380000
8,FERIA DE VEHICULO,17580000
9,FOMENTO CAMPAÑA EMPRESARIAL,14318000


Cree una tabla con la columna NumCre adicionando columna nueva (Saldo) donde se calcule el saldo por pagar utilizando las columnas ValPre y ValPag

In [15]:
df_new = amt[['NumCre', 'ValPre', 'ValPag']].copy()
df_new['Saldo'] = df_new['ValPre'] - df_new['ValPag']

columns_drop = ['ValPre', 'ValPag']

for i in columns_drop:
    df_new.drop([i], axis=1, inplace=True)

df_new.sample(n=5).head()

,NumCre,Saldo
5998,4410015914,1135064
9952,4090034583,209029
3002,4010022180,1369977
8596,4720021205,4040075
7503,4650030716,938822


In [16]:
df_new.to_sql(
        'Saldo',  # Table name
        connection,
        if_exists='replace',
        index=False
    )

connection.close()

## 1.b

Realice análisis sobre los datos utilizando las columnas EstCre, IndEst de la base consolidada. Apoyarse en herramientas de visualización tablas dinámicas y gráficos. Adjuntar evidencias.

In [17]:
amt['PERIODO'] = amt['PERIODO'].astype(str)

In [18]:
amt.groupby(['PERIODO', 'EstCre', 'IndEst']).size().to_frame()[0:20]

0
PERIODO EstCre    IndEst                
201508  Castigado 181-360 Dias         2
                  Mayor a 360 Dias   484
        Juridico  181-360 Dias         4
                  Mayor a 360 Dias    11
        Moroso    1-30 Dias          195
                  121-180 Dias        12
                  181-360 Dias        26
                  31-60 Dias          12
                  61-90 Dias          12
                  91-120 Dias          7
                  Mayor a 360 Dias     8
        Vigente   Vigente           1227
201509  Castigado 181-360 Dias         7
                  Mayor a 360 Dias   345
        Juridico  1-30 Dias            1
                  121-180 Dias         1
                  181-360 Dias         3
                  Mayor a 360 Dias     5
        Moroso    1-30 Dias          163
                  121-180 Dias         9

In [19]:
fig = px.bar(amt, x='PERIODO', y='EstCre', color='IndEst', title='Estado del crédito por periodo')
fig.show()

## 1.c

Realice un análisis con la información de la base consolidada sin utilizar variables del punto anterior Apoyarse en herramientas de visualización tablas dinámicas y gráficos. Adjuntar evidencias.

In [20]:
amt.groupby('PERIODO')['ValPre', 'ValPag'].sum()

C:\Users\almontao\AppData\Local\Temp\ipykernel_100\4212151089.py:1: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



,ValPre,ValPag
PERIODO,,
201508,8968671422,2722420049
201509,14573857471,4865087770
201510,11353207322,3943784043
201511,33703517567,18618476876
201512,8407501137,2743226014


In [21]:
fig = px.parallel_categories(amt, dimensions=['PERIODO', 'Origen de Fondos', 'Destino Fondos'])
fig.show()

## 1.d

Elabore un indicador de gestión que evidencie la participación de créditos de libre inversión sobre el monto total de crédito, con frecuencia mensual, resultado acumulado. Genere un gráfico que describa el comportamiento del resultado y elabore conclusiones. Apoyarse en herramientas de visualización tablas dinámicas y gráficos. Adjuntar evidencias.

In [22]:
df_new = amt.query("NomLin == 'LIBRE INVERSION'")

df_new['FecPrx'] = pd.to_datetime(df_new['FecPrx'], format='%Y%m%d')
# df_new['FecDsm'] = pd.to_datetime(df_new['FecDsm'], format='%Y%m%d')
df_new = df_new.set_index('FecPrx')

C:\Users\almontao\AppData\Local\Temp\ipykernel_100\2895981614.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
df_summary = pd.DataFrame()
# df_summary['promedio_semanal'] = df_new['FecPrx'].resample('M').mean().round(2)  # Creation of a new column with an average of the sum of cases per Month

In [24]:
df_summary.head()

""


# Optimization

In [25]:
# Delete variables to free memory
del df_CREDITO_201508, df_CREDITO_201509, df_CREDITO_201510
del df_CREDITO_201511, df_CREDITO_201512, df_new

# Second part

Cargar en una base de datos los archivos Productos, Ventas e Inventarios que se adjuntan, para todos los puntos adjuntar scripts, resultados y la evidencia que considere:

In [26]:
filepaths = [f for f in os.listdir(PATH + 'data') if f.endswith('.txt')]  # Get filenames

# Generate dataframe names
df_names = filepaths.copy()
for i in range(len(df_names)): df_names[i] = 'df_' + df_names[i].rstrip('.txt')

In [27]:
# Read multi-column fixed-width files
colspecs = [(0, 11), (12, 35), (36, 47), (48, 69), (70, 109), (110, 149), (150, 161)]  # define column widths
df_Inventario = pd.read_fwf(PATH + 'data/Inventario.txt', colspecs=colspecs)

colspecs = [(0, 11), (12, 62), (63, 113), (114, 123)]  # define column widths
df_Productos = pd.read_fwf(PATH + 'data/Productos.txt', colspecs=colspecs)

colspecs = [(0, 13), (14, 32), (33, 41), (42, 52), (53, 103), (104, 119), (120, 141), (142, 160), (161, 188)]  # define column widths
df_Ventas = pd.read_fwf(PATH + 'data/Ventas.txt', colspecs=colspecs)

In [28]:
df_Inventario['InventoryDate'] = pd.to_datetime(df_Inventario['InventoryDate'], format='%Y-%m-%d %H:%M:%S.%f')
df_Ventas['OrderDate'] = pd.to_datetime(df_Ventas['OrderDate'], format='%Y-%m-%d %H:%M:%S.%f')

In [29]:
connection = db.connect('Punto 1/INVENTARIO.db')

In [30]:
for i in range(len(df_names)):
    dataframe = df_names[i]
    table_name = df_names[i].lstrip('df_')
    globals()[dataframe].to_sql(  # Convert string into variable name with "globals()[variable_name]"
        table_name,
        connection,
        if_exists='replace',
        index=False
    )

In [31]:
connection.close()

## 2.a

En el archivo de productos, algunos tienen la talla de los artículos, esta talla se encuentra en el campo Name después de una coma (,) más un espacio (“ ”) y está representado por números de dos dígitos o por letras (una o dos), los que no cuenten con la talla, déjelos en blanco, del campo Name obtenga la talla de los artículos en una nueva columna que se llame Size.

In [32]:
df_Productos[3:13].head(10)

,ProductID,Name,ProductModel,CultureI
3,984,"Mountain-500 Silver, 40",Mountain-500,en
4,985,"Mountain-500 Silver, 42",Mountain-500,en
5,986,"Mountain-500 Silver, 44",Mountain-500,en
6,987,"Mountain-500 Silver, 48",Mountain-500,en
7,988,"Mountain-500 Silver, 52",Mountain-500,en
8,989,"Mountain-500 Black, 40",Mountain-500,en
9,990,"Mountain-500 Black, 42",Mountain-500,en
10,993,"Mountain-500 Black, 52",Mountain-500,en
11,980,"Mountain-400-W Silver, 38",Mountain-400-W,en
12,981,"Mountain-400-W Silver, 40",Mountain-400-W,en


In [33]:
df_Productos[['Name', 'Size']] = df_Productos['Name'].str.split(',', expand=True)

In [34]:
df_Productos[3:13].head(10)

,ProductID,Name,ProductModel,CultureI,Size
3,984,Mountain-500 Silver,Mountain-500,en,40
4,985,Mountain-500 Silver,Mountain-500,en,42
5,986,Mountain-500 Silver,Mountain-500,en,44
6,987,Mountain-500 Silver,Mountain-500,en,48
7,988,Mountain-500 Silver,Mountain-500,en,52
8,989,Mountain-500 Black,Mountain-500,en,40
9,990,Mountain-500 Black,Mountain-500,en,42
10,993,Mountain-500 Black,Mountain-500,en,52
11,980,Mountain-400-W Silver,Mountain-400-W,en,38
12,981,Mountain-400-W Silver,Mountain-400-W,en,40


In [35]:
df_Productos['Size'].value_counts()

 44       173
 48       149
 52        96
 58        90
 42        90
 38        72
 40        66
 46        66
 60        66
 62        66
 M         66
 L         66
 54        54
 50        54
 S         54
 XL        18
 56        12
 Large      6
 Red        6
 Black      6
 Blue       6
Name: Size, dtype: int64

## 2.b

Del campo ProductModel según el texto encontrado en el archivo de productos extraiga la clasificación del producto en una nueva columna y llámela ProductType, estas pueden ser: Mountain, Road, Touring, Classic o Ninguno (Para los que no contengan ninguna de esas categorías).

In [36]:
df_Productos['ProductType'] = df_Productos['ProductModel'].apply(lambda x: get_product_type(x))
df_Productos.sample(n=15).head(15)

,ProductID,Name,ProductModel,CultureI,Size,ProductType
1019,813,HL Road Handlebars,HL Road Handlebars,th,None,Ninguno
20,780,Mountain-200 Silver,Mountain-200,en,42,Mountain
208,719,HL Road Frame - Red,HL Road Frame,en,48,Ninguno
674,968,Touring-1000 Blue,Touring-1000,fr,54,Touring
143,818,LL Road Front Wheel,LL Road Front Wheel,en,None,Ninguno
370,963,Touring-3000 Yellow,Touring-3000,ar,54,Touring
1444,995,ML Bottom Bracket,ML Bottom Bracket,zh-cht,None,None
1360,731,ML Road Frame - Red,ML Road Frame,he,44,Ninguno
698,906,ML Mountain Frame-W - Silver,ML Mountain Frame-W,fr,46,Ninguno
1574,743,HL Mountain Frame - Black,HL Mountain Frame,zh-cht,42,Ninguno


In [37]:
df_Productos['ProductType'].value_counts()

Ninguno     768
Road        270
Mountain    220
Touring     168
Classic      18
Name: ProductType, dtype: int64

## 2.c

Realice un cruce entre las 3 tablas y genere un reporte donde identifique: 
- los productos que están en la tabla de ventas y que no se encuentran en la tabla de productos
- los productos que están en la tabla de productos y que no se encuentran en la tabla de Inventarios
- los productos que están en la tabla de ventas y que no se encuentran en la tabla de inventarios

Adjuntar evidencia del cruce y explíquelo.

In [38]:
df_new = df_Ventas.merge(df_Productos, how='outer', indicator='Unión')
df_new[df_new['Unión'] == 'left_only']
df_new['Unión'].value_counts()

left_only     121318
right_only      1762
both               0
Name: Unión, dtype: int64

In [39]:
df_new = df_Productos.merge(df_Inventario, how='outer', indicator='Unión')
df_new[df_new['Unión'] == 'left_only']
df_new['Unión'].value_counts()

left_only     1270
both           492
right_only     183
Name: Unión, dtype: int64

In [40]:
df_new = df_Ventas.merge(df_Inventario, how='outer', indicator='Unión')
df_new[df_new['Unión'] == 'left_only']
df_new['Unión'].value_counts()

left_only     121318
right_only       265
both               0
Name: Unión, dtype: int64